In [8]:
import pymysql
import pandas as pd

In [ ]:
import pymysql
import pandas as pd

# Correct DataFrame (no duplicate column names)
df = pd.DataFrame({
    'sepal_length': [5.1, 4.9, 4.7],
    'sepal_width': [3.5, 3.0, 3.2],
    'petal_length': [1.4, 1.4, 1.3],
    'petal_width': [0.2, 0.2, 0.2],
    'species': ['setosa', 'setosa', 'setosa']
})

try:
  # Connect to MariaDB
  conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    port=3306,
    password='NewStrongPasswordHere',
    database='cleaned_data_database_2'
  )
  
  with conn.cursor() as cursor:
    # Create table
    cursor.execute("""
      CREATE TABLE IF NOT EXISTS Iris (
        sepal_length FLOAT,
        sepal_width FLOAT,
        petal_length FLOAT,
        petal_width FLOAT,
        species VARCHAR(20)
      )
    """)

    # Insert data
    insert_query = """
      INSERT INTO Iris (sepal_length, sepal_width, petal_length, petal_width, species)
      VALUES (%s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_query, df.values.tolist())

  conn.commit()
  print("✅ Data inserted successfully.")

except Exception as e:
  print("❌ Error:", e)

finally:
  if conn:
    conn.close()
    print("🔒 Connection closed.")


❌ Error: (2003, "Can't connect to MySQL server on 'main-mariadb' ([Errno -3] Temporary failure in name resolution)")


Error: Already closed

In [ ]:
# Retrieve the data back to verify
import pymysql
import pandas as pd

# Connect to MariaDB
conn = pymysql.connect(
    host='localhost',
    user='birat-gautam',
    password='NewStrongPasswordHere',
    database='scrapped_data_database_1'
)

# Use pandas to read the SQL table into a DataFrame
df_retrieved = pd.read_sql("SELECT * FROM Iris", conn)

conn.close()

# Show the DataFrame
df_retrieved.head()


/tmp/ipykernel_69055/2475506504.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retrieved = pd.read_sql("SELECT * FROM Iris", conn)


sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa

In [13]:
import pandas as pd
import pymysql

def generate_create_table_sql(df, table_name="final_dataset"):
    dtype_mapping = {
        'int64': 'INT',
        'float64': 'DOUBLE',
        'object': 'VARCHAR(255)',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'DATETIME',
    }

    columns = []
    for col, dtype in df.dtypes.items():
        col_type = dtype_mapping.get(str(dtype), 'VARCHAR(255)')
        if col == df.columns[0] and col_type in ['INT', 'BIGINT']:
            columns.append(f"{col} {col_type} PRIMARY KEY")
        else:
            columns.append(f"{col} {col_type}")
    
    columns_sql = ",\n    ".join(columns)
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n    {columns_sql}\n);"
    return create_table_sql

def main():
    # Load the DataFrame
    df = pd.read_csv("./transformed_data_final.csv")
    table_name = "final_dataset"

    # Generate SQL to create the table
    create_table_sql = generate_create_table_sql(df, table_name)

    # Database credentials
    db_config = {
        "host": "localhost",
        "user": "birat-gautam",
        "password": "NewStrongPasswordHere",
        "database": "scrapped_data_database_1",
        "port": 3306
    }

    try:
        # Connect to the database
        conn = pymysql.connect(**db_config)
        cursor = conn.cursor()

        # Create the table
        cursor.execute(create_table_sql)
        print(f"✅ Table `{table_name}` created or already exists.")

        # Prepare insert query
        placeholders = ", ".join(["%s"] * len(df.columns))
        insert_query = f"""
            INSERT INTO {table_name} ({', '.join(df.columns)})
            VALUES ({placeholders})
        """

        # Insert data
        cursor.executemany(insert_query, df.values.tolist())
        conn.commit()
        print(f"✅ Inserted {len(df)} rows into `{table_name}`")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()
            print("🔒 Connection closed.")

if __name__ == "__main__":
    main()


✅ Table `final_dataset` created or already exists.
✅ Inserted 7260 rows into `final_dataset`
🔒 Connection closed.
